task name: LoadingSvmLight
task description: Implement the LoadingSvmLight.Example() method to load data with SvmLightLoader.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.IO;
using System.Text;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
// Create a random SVM light format file.
var random = new Random(42);
var dataDirectoryName = "DataDir";
Directory.CreateDirectory(dataDirectoryName);
var fileName = Path.Combine(dataDirectoryName, $"SVM_Data.csv");
using (var fs = File.CreateText(fileName))
{
    // Write random lines in SVM light format
    for (int line = 0; line < 10; line++)
    {
        var sb = new StringBuilder();
        if (random.NextDouble() > 0.5)
            sb.Append("1 ");
        else
            sb.Append("-1 ");
        if (line % 2 == 0)
            sb.Append("cost:1 ");
        else
            sb.Append("cost:2 ");
        for (int i = 1; i <= 10; i++)
        {
            if (random.NextDouble() > 0.5)
                continue;
            sb.Append($"{i}:{random.NextDouble()} ");
        }
        fs.WriteLine(sb.ToString());
    }
}

// Create MLContext and MultiFileSource
var mlContext = new MLContext();
var file = new MultiFileSource(fileName);

In [ ]:
var loader = mlContext.Data.CreateSvmLightLoader(dataSample: file);
var svmData = loader.Load(file);

In [ ]:
void PrintSchema(IDataView data)
{
    foreach (var col in data.Schema)
        Console.WriteLine($"Column {col.Name} type {col.Type}");
}

void PrintData(IDataView data)
{
    using (var cursor = data.GetRowCursor(data.Schema))
    {
        var labelGetter = cursor.GetGetter<float>(data.Schema["Label"]);
        var weightGetter = cursor.GetGetter<float>(data.Schema["Weight"]);
        var featuresGetter = cursor.GetGetter<VBuffer<float>>(data.Schema["Features"]);

        VBuffer<float> features = default;
        while (cursor.MoveNext())
        {
            float label = default;
            labelGetter(ref label);

            float weight = default;
            weightGetter(ref weight);

            featuresGetter(ref features);

            Console.WriteLine($"{label} {weight} {string.Join(' ', features.DenseValues())}");
        }
    }
}

PrintSchema(svmData);
PrintData(svmData);

In [ ]:
loader = mlContext.Data.CreateSvmLightLoader(inputSize: 10);
svmData = loader.Load(file);

In [ ]:
PrintSchema(svmData);
PrintData(svmData);